In [1]:
import matplotlib
matplotlib.use('Agg')

import cv2
import scipy
import pprint
import random
import itertools
import numpy as np
from sklearn.svm import SVC
from sklearn import datasets
from collections import Counter
from skimage.feature import hog
from sklearn import preprocessing
from skimage import data
import keras
from sklearn.externals import joblib

Using TensorFlow backend.


In [2]:
from matplotlib import pyplot as plt

%matplotlib auto

Using matplotlib backend: TkAgg


In [3]:
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original')
data = mnist.data
target = mnist.target

In [4]:
len(data)

70000

In [5]:
def plot_things(things, labels):
    count = len(things)
    plt.figure(figsize=(20, 20))
    
    plt.axis('off')
    plt.xticks([])
    plt.yticks([])
    for i, thing in enumerate(things):
        cols = 10
        rows = int(count / cols) + 1
        ax = plt.subplot(rows, cols, 1 + i)
        
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_title(labels[i])

        plt.imshow(thing)
    plt.show()

# Generating 2-digit combinations

In [19]:
import random
from skimage.filters import threshold_otsu

def random_combination(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.sample(xrange(n), r))
    return tuple(pool[i] for i in indices)

def remove_border(img):
    where = np.where(img > 0)
    y1 = np.min(where[0])
    y2 = np.max(where[0])
    x1 = np.min(where[1])
    x2 = np.max(where[1])
    crop = img[y1:y2 , x1:x2]
    return crop

n_samples = 1000000
combs = []
combs_labels = []
data_w_labels = zip(data, target)
random.shuffle(data_w_labels)

for i in range(n_samples):
    comb = random_combination(data_w_labels, 2)
    
    first_digit = comb[0][0].reshape((28, 28))
    second_digit = comb[1][0].reshape((28, 28))

#     thresh = threshold_otsu(first_digit)
#     first_digit = (first_digit > thresh).astype(np.uint8)
    
#     thresh = threshold_otsu(second_digit)
#     second_digit = (second_digit > thresh).astype(np.uint8)
    
    first_mod = remove_border(first_digit)
    second_mod = remove_border(second_digit)
    
    # Make sure height diff is divisible by 2
    if not (first_mod.shape[0] % 2 == 0):
        first_mod = np.vstack([first_mod, np.zeros((1, first_mod.shape[1]))])
    if not (second_mod.shape[0] % 2 == 0):
        second_mod = np.vstack([second_mod, np.zeros((1, second_mod.shape[1]))])
    
    height_diff = first_mod.shape[0] - second_mod.shape[0]
    
    if height_diff < 0:
        padding = int(-height_diff / 2.0)
        thepad = np.zeros((padding, first_mod.shape[1]))
        first_mod = np.vstack([first_mod, thepad])
        first_mod = np.vstack([thepad, first_mod])
    elif height_diff > 0:
        padding = int(height_diff / 2.0)
        thepad = np.zeros((padding, second_mod.shape[1]))
        second_mod = np.vstack([second_mod, thepad])
        second_mod = np.vstack([thepad, second_mod])
    
    # Align width
    overlap = 1
    
    height = first_mod.shape[0]
    width_1 = first_mod.shape[1]
    width_2 = second_mod.shape[1]
    
    padding = np.zeros((height, width_2 - overlap))
    first_mod = np.hstack([first_mod, padding])
    padding = np.zeros((height, width_1 - overlap))
    second_mod = np.hstack([padding, second_mod])
    
    overlapped = first_mod.astype(np.uint64) + second_mod.astype(np.uint64)
    overlapped = np.clip(overlapped, 0, 255).astype(np.uint8)
    padded = np.pad(overlapped, (5, 5), 'constant', constant_values=(0, 0))  
    
#     binary = (padded > 0).astype(np.uint8)
    combs.append(padded)
    
    label = int(  str(int(comb[0][1])) + str(int(comb[1][1]))  )
    combs_labels.append( label )
    
# combs = np.array(combs)
combs_labels = np.array(combs_labels)

In [20]:
combs_reshaped = [cv2.resize(x, (38, 28)) for x in combs]

In [21]:
len(combs_reshaped)

1000000

In [18]:
i = 50
plot_things(combs_reshaped[i:i + 50], combs_labels[i:i + 50])

# Train Classifier

In [ ]:
from sklearn.cross_validation import train_test_split
from keras.utils.np_utils import to_categorical

# Extract the features and labels
combs_np = np.array(combs_reshaped)
features = np.expand_dims(combs_np.reshape(1000000, 28, 38), axis=3)
labels = to_categorical(combs_labels)

x_train, y_train, x_test, y_test = train_test_split(features, labels)

In [23]:
import keras
from keras import applications
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


def get_model():
    model = Sequential()

    model.add(Conv2D(32, (3,3), input_shape=(28, 38, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(Activation('softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model


batch_size = 512
num_classes = 10
epochs = 7

model = get_model()

In [ ]:
earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

history_bn = model.fit(
    features,
    labels,
    validation_split=0.05,
    batch_size=batch_size,
    callbacks=[earlyStopping], 
    epochs=epochs
)

# Save the classifier
model.save(filepath='./double_digits_cnn_nonbinary')

Train on 950000 samples, validate on 50000 samples
Epoch 1/7
950000/950000 [==============================] - 333s - loss: 0.5841 - acc: 0.8435 - val_loss: 0.0759 - val_acc: 0.9774
Epoch 2/7
950000/950000 [==============================] - 330s - loss: 0.1171 - acc: 0.9651 - val_loss: 0.0477 - val_acc: 0.9854
Epoch 3/7
950000/950000 [==============================] - 331s - loss: 0.0802 - acc: 0.9759 - val_loss: 0.0396 - val_acc: 0.9882
Epoch 4/7
950000/950000 [==============================] - 330s - loss: 0.0631 - acc: 0.9808 - val_loss: 0.0291 - val_acc: 0.9907
Epoch 5/7
950000/950000 [==============================] - 330s - loss: 0.0523 - acc: 0.9838 - val_loss: 0.0257 - val_acc: 0.9922
Epoch 6/7
950000/950000 [==============================] - 330s - loss: 0.0450 - acc: 0.9861 - val_loss: 0.0237 - val_acc: 0.9925
Epoch 7/7
879104/950000 [==========================>...] - ETA: 24s - loss: 0.0397 - acc: 0.9877

# Test on Gels

In [12]:
clf, pp = joblib.load("digits_cls.pkl")
model = keras.models.load_model('./double_digits_cnn_nonbinary')
model_single = keras.models.load_model('./digits_cnn')

In [5]:
def sort_rects(rects_ctrs):
    rects_sort = sorted(rects_ctrs, key=lambda r: r[0][0])
    
    result = []
    for rect in rects_sort:
        # Filter to only rects with overlap of > 1/3 of target height
        rects_sort_filt = filter(lambda x: calc_overlap(x[0], rect[0]) > (x[0][3] / 3.0) and abs(x[0][3] - rect[0][3]) < (x[0][3]), rects_sort)
        rects_sort_filt = sorted(rects_sort_filt, key=lambda r: r[0][0])
        result += rects_sort_filt
        for rect_sorted in rects_sort_filt:
            rects_sort.remove(rect_sorted)
        
    return zip(*result)
    

def calc_overlap(rect_1, rect_2):
    rect_1_x = rect_1[0]
    rect_1_y = rect_1[1]
    rect_1_width = rect_1[2]
    rect_1_height = rect_1[3]
    
    rect_2_x = rect_2[0]
    rect_2_y = rect_2[1]
    rect_2_width = rect_2[2]
    rect_2_height = rect_2[3]
    
    overlap = min(rect_1_y + rect_1_height, rect_2_y + rect_2_height) - max(rect_1_y, rect_2_y)
    return overlap

In [129]:
def extract_numbers(im, thresh=30, blur=False):
    im_one = im.astype(np.uint8)[:,:,2]

    # Convert to grayscale
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    # Invert the image
    im_gray = (255 - im_one)
    
    # Optional Gaussian blur
    if blur:
        im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

    # Threshold the image
    ret, im_th_all_colors = cv2.threshold(im_gray, thresh, 255, cv2.THRESH_BINARY)
    
    # Or try with specific threshold
    lower, upper = boundaries['black']
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
    mask = cv2.inRange(im, lower, upper)
    im_th = cv2.bitwise_and(im, im, mask = mask).astype(np.uint8)
    im_th = cv2.cvtColor(im_th, cv2.COLOR_BGR2GRAY)

    # Find contours in the image
    _, ctrs, hierarchy = cv2.findContours(im_th, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    ctrs = [c for i, c in enumerate(ctrs) if hierarchy[0][i][-1] == -1]

    # Get contour bounding boxes
    rects = [cv2.boundingRect(ctr) for ctr in ctrs]
    
    # Sorted order of rois
    sorted_rects, sorted_ctrs = sort_rects(zip(rects, ctrs))

    rois = []
    nbrs = []
    probs = []
    prev_not_one_width = 99999
    date_possibs = []
    cur_date_possib = []
    prev_end_x = sorted_rects[0][0] + sorted_rects[0][2]
    prev_end_y = sorted_rects[0][1] + sorted_rects[0][3]
    
    # For each rectangular region, predict the digit using classifier
    for i, rect in enumerate(sorted_rects):
        x_start = rect[0]
        y_start = rect[1]
        width = rect[2]
        height = rect[3]

        # Skip short artifacts
        if height < 10: continue
        # Skip long artifacts
        if width > 100: continue

        im_th_all_colors = im_th_all_colors.astype(np.float64)
        im_roi = im_th_all_colors[y_start : y_start + height, x_start : x_start + width]

        # Draw filled contours (removes overlapping items)
        mask = np.zeros((height, width)).astype(np.uint8)
        mask = cv2.drawContours(mask, sorted_ctrs, i, (255, 255, 255), cv2.FILLED, offset=(-x_start, -y_start))
        roi = cv2.bitwise_and(mask.astype(np.uint8), im_roi.astype(np.uint8)).astype(np.uint8)

        # Pad ROI to square
        if height < width:
            padding = int((width - height) / 2.0)
            roi_pad = np.pad(roi, (padding, padding), 'constant', constant_values=(0, 0))
        elif width < height:
            padding = int((height - width) / 2.0)
            roi_pad = np.pad(roi, (padding, padding), 'constant', constant_values=(0, 0))
        else:
            roi_pad = roi
        roi_pad = np.pad(roi_pad, (5, 5), 'constant', constant_values=(0, 0))
        
        # Differentiate single from connected digits
        if width > 1.25 * prev_not_one_width:
            roi_resized = cv2.resize(roi_pad, (38 ,28), interpolation=cv2.INTER_NEAREST)
            roi_cnn = np.expand_dims(roi_resized, axis=2)

            prob = model.predict_proba(np.array([roi_cnn]), verbose=0)
            nbr = np.argmax(prob)
            nbr_prob = prob[0]
        else:
            roi_resized = cv2.resize(roi_pad, (28 ,28), interpolation=cv2.INTER_NEAREST)
            roi_cnn = np.expand_dims(roi_resized, axis=2)

            prob = model_single.predict_proba(np.array([roi_cnn]), verbose=0)
            nbr = np.argmax(prob)
            nbr_prob = prob[0]

        # Add nbr to list, or create new date possib (for newlines)
        if x_start - prev_end_x < 80 and y_start - prev_end_y < 80:
            cur_date_possib.append(nbr)
        else:
            date_possibs.append(cur_date_possib)
            cur_date_possib = [nbr]

        prev_end_x = x_start
        prev_end_y = y_start

        # Mark the roi and label
        cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 100, 255), 1)
        cv2.putText(im, str(int(nbr)), (rect[0], rect[1]), cv2.FONT_ITALIC, 0.4, (0, 0, 255), 1)
            
        probs.append(0)
        rois.append(roi_resized)
        nbrs.append(nbr)   
        
        if nbr != 1:
            prev_not_one_width = width
       
    # Append last date possibility
    date_possibs.append(cur_date_possib)
    
    plt.imshow(im)
    plt.show()
    
    return rois, nbrs, date_possibs, probs

In [134]:
imgs = [1,6,12,21,41,42,51,52,56,83,84,89,90,96,97,106,123,131,136,152,153,156,157] #7, 22
filenames = ['../data/gels_nov_2016/Im{} - p. {}.png'.format(i, i) for i in imgs]
images = [scipy.misc.imread(f) for f in filenames]

boundaries = {
#     ([30, 15, 20], [50, 40, 60]), # black ink
    'black': ([0, 0, 0], [190, 190, 190]), # black ink
    'blue': ([50, 31, 4], [85, 120, 180]), # blue dye
}

colors = []
image = scipy.misc.imread(filenames[0])
for bound in boundaries.keys():
    lower, upper = boundaries[bound]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")

    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask = mask)
    colors.append(output)   

# im = scipy.misc.imread(filenames[8])#[580:880,440:980]   
# roi = im[790:880,520:750]
# imgs = [roi]

# im = scipy.misc.imread(filenames[8])
# roi = im[800:880,500:800]
# imgs = [roi]

results = [extract_numbers(i) for i in images[6:7]]

In [76]:
filenames[9:10]

['../data/gels_nov_2016/Im83 - p. 83.png']